In [1]:
import glob
import pandas as pd
import numpy as np
import cv2
from skimage import feature
import pickle

In [2]:
# Load Data
data = pd.read_csv('x_y.csv')


In [3]:
# Split into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(data),data[['y0','y1','y2','y3','y4','y5']], test_size=.2, random_state=42)

In [4]:
batch_size = 100
x_pixels = int(100)
y_pixels = int(100)
n_classes = 6

In [5]:
def get_data(data_x,data_y,i):
    i = i*batch_size
    x_data_cnn = []
    x_data_neural = []
       
    type_class_check = type(np.array([1,2,3]))
    for counter in range(i,i+batch_size):
        try:
            #Insert Path to file where all images are present
            image = cv2.imread('/home/pratik/Desktop/be_project/images/Data/Images/working/'+data_x.iloc[counter]['name']+'.jpg',cv2.IMREAD_GRAYSCALE)
            if type(image) == type_class_check:
                x_data_cnn.append(cv2.resize(image,(x_pixels,y_pixels)))
                
            else:
                #print('nai hai')
                x_data_cnn.append(np.zeros([x_pixels,y_pixels]))
            x_data_neural.append(data_x.iloc[counter][['age','sex','s0','s1','s2','s3','s4','s5','s6']])
        except ValueError:
            print("Missed data point")
            
    y_data = data_y[['y0','y1','y2','y3','y4','y5']][i:i+batch_size]
    #x_data = np.zeros([batch_size,x_pixels,y_pixels])
    return x_data_cnn,x_data_neural, y_data


In [6]:
import tensorflow as tf

n_classes = 6
batch_size = 100

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

#The place holder to hold the values of new x and y at different times.
x_neural = tf.placeholder('float', [batch_size, 9])
x_cnn = tf.placeholder('float', [batch_size, x_pixels*y_pixels])
y = tf.placeholder('float',[batch_size,n_classes])

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([9, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    #output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']
    return l3


def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([(int(x_pixels//4))*(int(y_pixels//4))*64,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, x_pixels, y_pixels, 1])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, (int(x_pixels//4))*(int(y_pixels//4))*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    #print(fc.shape)
    #output = tf.matmul(fc, weights['out'])+biases['out']
    #print(output.shape)
    return fc






def train_neural_network(x_neural,x_cnn):
    output_combined = {'weights':tf.Variable(tf.random_normal([1524, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}
    output_cnn = convolutional_neural_network(x_cnn)
    output_neural = neural_network_model(x_neural)
    print(output_cnn.shape,output_neural.shape)
    output = tf.concat([output_cnn,output_neural],1)
    print(output.shape)
    prediction = tf.add(tf.matmul(output,output_combined['weights']), output_combined['biases'])
    #prediction = tf.nn.relu(raw_pred)
    #prediction = neural_network_model(x_neural)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
    
    hm_epochs = 5
    
    
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        import datetime

        tf.summary.scalar('Loss', cost)
        tf.summary.scalar('Accuracy', accuracy)
        merged = tf.summary.merge_all()
        logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
        writer = tf.summary.FileWriter(logdir, sess.graph)
        for epoch in range(hm_epochs):
            epoch_loss = 0
            epoch_accuracy = 0
            for i in range(len(X_train)//batch_size):
                print((i*10000)/len(X_train))
                epoch_x_cnn,epoch_x_neural, epoch_y = get_data(X_train,y_train,i)
                epoch_x_cnn = np.array(epoch_x_cnn).reshape(batch_size,x_pixels*y_pixels)
                _, c, a = sess.run([optimizer, cost,accuracy], feed_dict={x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                #o = sess.run([cost], feed_dict={x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                #print(f.shape, o.shape)
                epoch_loss += c
                epoch_accuracy += a
                print("current accuracy: "+str(a)+"current loss: "+str(c))
                summary = sess.run(merged, {x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                writer.add_summary(summary, i)
                
                
        
                
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        
        test_x_cnn, test_x_neural, test_y = get_data(X_test,y_test,1)
        test_x_cnn = np.array(test_x_cnn).reshape(batch_size,x_pixels*y_pixels)
        print('Accuracy:',sess.run([accuracy],feed_dict={x_cnn: test_x_cnn, x_neural: test_x_neural, y: test_y}))
        
train_neural_network(x_neural,x_cnn)



/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(100, 1024) (100, 500)
(100, 1524)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.0
current accuracy: 0.2current loss: 19209.37
0.584180394905947
current accuracy: 0.25current loss: 9556.19
1.168360789811894
current accuracy: 0.4current loss: 7141.805
1.752541184717841
current accuracy: 0.54current loss: 7211.7476
2.336721579623788
current accuracy: 0.58current loss: 50291.695
2.920901974529735
current accuracy: 0.63current loss: 4885.02
3.505082369435682
current accuracy: 0.72current loss: 4557.4785
4.089262764341629
current accuracy: 0.69current loss: 3212.5505
4.673443159247576
current accuracy: 0.67current loss: 2790.1335
5.257623554153523
current accuracy: 0.7current loss: 2426.1877
5.84180394905947
current accuracy: 0.72current loss: 2017.9526
6.425984343965417
curr

current accuracy: 1.0current loss: 0.0
71.27000817852553
current accuracy: 1.0current loss: 0.0
71.85418857343147
current accuracy: 0.98current loss: 111.65604
72.43836896833743
current accuracy: 0.99current loss: 31.141773
73.02254936324337
current accuracy: 0.97current loss: 102.86973
73.60672975814931
current accuracy: 1.0current loss: 0.0
74.19091015305527
current accuracy: 1.0current loss: 0.0
74.77509054796121
current accuracy: 0.99current loss: 2.469619
75.35927094286716
current accuracy: 0.99current loss: 13.562256
75.9434513377731
current accuracy: 1.0current loss: 0.0
76.52763173267905
current accuracy: 0.98current loss: 49.76626
77.111812127585
current accuracy: 1.0current loss: 0.0
77.69599252249094
current accuracy: 1.0current loss: 0.0
78.2801729173969
current accuracy: 0.99current loss: 20.43647
78.86435331230284
current accuracy: 1.0current loss: 0.0
79.44853370720878
current accuracy: 0.99current loss: 1.399237
80.03271410211474
current accuracy: 0.98current loss: 51.7

current accuracy: 1.0current loss: 0.0
53.160415936441176
current accuracy: 1.0current loss: 0.0
53.74459633134712
current accuracy: 1.0current loss: 0.0
54.32877672625307
current accuracy: 1.0current loss: 0.0
54.91295712115901
current accuracy: 1.0current loss: 0.0
55.49713751606496
current accuracy: 1.0current loss: 0.0
56.08131791097091
current accuracy: 1.0current loss: 0.0
56.66549830587685
current accuracy: 1.0current loss: 0.0
57.2496787007828
current accuracy: 0.99current loss: 12.68883
57.83385909568875
current accuracy: 1.0current loss: 0.0
58.418039490594694
current accuracy: 1.0current loss: 0.0
59.002219885500644
current accuracy: 1.0current loss: 0.0
59.586400280406586
current accuracy: 1.0current loss: 0.0
60.170580675312536
current accuracy: 1.0current loss: 0.0
60.754761070218485
current accuracy: 1.0current loss: 0.0
61.33894146512443
current accuracy: 1.0current loss: 0.0
61.92312186003038
current accuracy: 1.0current loss: 0.0
62.50730225493633
current accuracy: 1.

KeyboardInterrupt: 